# Executive Summary
***

This notebook pre-processes the weather data from Newark Liberty International Airport. 

The original dataset is one table containing different weather measurements for each day of 2016. (First 5 rows are shown below). The dataset is of high quality with few missing data points, no duplicated columns and no data integrity issues. 

In this notebook, the data is checked for sanity and validated, missing data is identified and filled in and units are converted. No new columns are added. The resulting table is exported as a .csv file to be uploaded into a PostgreSQL database. 

# Importing Data and Packages
***

In [6]:
import pandas as pd

In [7]:
weather_data = pd.read_csv('C:/Users/Dell/OneDrive/Dokumenty/Data Engineering/Codecademy Course/Bike Rental Project/bike-rental-starter-kit/data/newark_airport_2016.csv')
weather_df = pd.DataFrame(weather_data)
weather_df.head(5)

,STATION,NAME,DATE,AWND,PGTM,PRCP,SNOW,SNWD,TAVG,TMAX,TMIN,TSUN,WDF2,WDF5,WSF2,WSF5
0,USW00014734,"NEWARK LIBERTY INTERNATIONAL AIRPORT, NJ US",2016-01-01,12.75,NaN,0.0,0.0,0.0,41,43,34,NaN,270,280.0,25.9,35.1
1,USW00014734,"NEWARK LIBERTY INTERNATIONAL AIRPORT, NJ US",2016-01-02,9.40,NaN,0.0,0.0,0.0,36,42,30,NaN,260,260.0,21.0,25.1
2,USW00014734,"NEWARK LIBERTY INTERNATIONAL AIRPORT, NJ US",2016-01-03,10.29,NaN,0.0,0.0,0.0,37,47,28,NaN,270,250.0,23.9,30.0
3,USW00014734,"NEWARK LIBERTY INTERNATIONAL AIRPORT, NJ US",2016-01-04,17.22,NaN,0.0,0.0,0.0,32,35,14,NaN,330,330.0,25.9,33.1
4,USW00014734,"NEWARK LIBERTY INTERNATIONAL AIRPORT, NJ US",2016-01-05,9.84,NaN,0.0,0.0,0.0,19,31,10,NaN,360,350.0,25.1,31.1


## Data Legend
***

The following table explains the names of the columns:
<br>**AWND** - Average daily wind speed
<br>**PGTM** - Peak gust time
<br>**PRCP** - Precipitation
<br>**SNOW** - Snowfall
<br>**SNWD** - Snow depth
<br>**TAVG** - Average temperature
<br>**TMAX** - Maximum temperature
<br>**TMIN** - Minimum temperature
<br>**TSUN** - Daily total sunshine
<br>**WDF2** - Direction of fastest 2-minute wind
<br>**WDF5** - Direction of fastest 5-second wind
<br>**WSF2** - Fastest 2-minute wind speed
<br>**WSF5** - Fastest 5-second wind speed

## Initial Exploration of the Data
***

In [8]:
print(weather_df['STATION'].nunique())
print(weather_df['NAME'].nunique())

1
1


In [9]:
print(weather_df['DATE'].min())
print(weather_df['DATE'].max())

2016-01-01
2016-12-31


In [10]:
print(weather_df['DATE'].count())

366


The dataset spans from the start of 2016 to the end of it. 
It contains data only from Newark airport. Information on every day is included (The year 2016 had 366 days).
The first two columns are dropped as they provide no additional information.

In [11]:
weather_df = weather_df.drop(['STATION', 'NAME'], axis = 1)

## Checking for missing and duplicated Data
***

In [12]:
weather_missing_values = weather_df.isna().sum().reset_index()
number_of_rows = weather_df['DATE'].count()
weather_missing_values = weather_missing_values.rename(columns = {'index' : 'column', 0 : 'number_of_missing_rows'})
weather_missing_values['percentage_missing'] =  100.0 * weather_missing_values['number_of_missing_rows'] / number_of_rows
print(weather_missing_values)
print('\nThe number of duplicated rows is:')
print(weather_df.duplicated().sum())

   column  number_of_missing_rows  percentage_missing
0    DATE                       0            0.000000
1    AWND                       0            0.000000
2    PGTM                     366          100.000000
3    PRCP                       0            0.000000
4    SNOW                       0            0.000000
5    SNWD                       0            0.000000
6    TAVG                       0            0.000000
7    TMAX                       0            0.000000
8    TMIN                       0            0.000000
9    TSUN                     366          100.000000
10   WDF2                       0            0.000000
11   WDF5                       2            0.546448
12   WSF2                       0            0.000000
13   WSF5                       2            0.546448

The number of duplicated rows is:
0


The dataset in general is in a good shape. 
The columns PGTM and TSUN will be dropped as they are completely empty.
There are two missing values in the WDF5 and WSF5 columns (These are the direction of the wind in degrees and the fastest 5 second wind respectively.) 
As is the common practice this missing data is at first assumed to be MNAR but this can not be falsified or approved since the dataset contains no metadata or other information. 
The data is considered to be MCAR and the missing values are filled with mean of the column. This will introduce insignificant bias into the dataset and will preserve other values needed for analysis. 

In [13]:
weather_df = weather_df.drop(['PGTM', 'TSUN'], axis = 1)
weather_df['WDF5'].fillna(weather_df['WDF5'].mean(), inplace = True)
weather_df['WSF5'].fillna(weather_df['WSF5'].mean(), inplace = True)

## Unit Determination, Conversion and Rounding
***
Using documentation provided, the units are converted, included in the column names and rounded.

In [14]:
weather_df['AWND'] = weather_df['AWND'] * 0.44704
weather_df['PRCP'] = weather_df['PRCP'] * 25.4
weather_df['SNOW'] = weather_df['SNOW'] * 25.4
weather_df['SNWD'] = weather_df['SNWD'] * 25.4

weather_df['TAVG'] = (weather_df['TAVG'] - 32) / 1.8
weather_df['TMAX'] = (weather_df['TMAX'] - 32) / 1.8
weather_df['TMIN'] = (weather_df['TMIN'] - 32) / 1.8

weather_df['WSF2'] = weather_df['WSF2'] * 0.44704
weather_df['WSF5'] = weather_df['WSF5'] * 0.44704

weather_df = weather_df.rename(columns = {
    'DATE' : 'date',
    'AWND' : 'awnd_m/s',
    'PRCP' : 'prcp_mm',
    'SNOW' : 'snow_mm',
    'SNWD' : 'snwd_mm',
    'TAVG' : 'tavg_degc',
    'TMAX' : 'tmax_degc',
    'TMIN' : 'tmin_degc',
    'WDF2' : 'wdf2_deg',
    'WDF5' : 'wdf5_deg',
    'WSF2' : 'wsf2_m/s',
    'WSF5' : 'wsf5_m/s' 
})

columns_to_round_decimal = ['awnd_m/s', 'prcp_mm', 'snow_mm', 'snwd_mm', 'tavg_degc', 'tmax_degc', 'tmin_degc', 'wdf2_deg', 'wdf5_deg', 'wsf2_m/s', 'wsf5_m/s']
weather_df[columns_to_round_decimal] = weather_df[columns_to_round_decimal].round(1)
weather_df['wdf5_deg'] = weather_df['wdf5_deg'].astype(int)
weather_df.head(5)

,date,awnd_m/s,prcp_mm,snow_mm,snwd_mm,tavg_degc,tmax_degc,tmin_degc,wdf2_deg,wdf5_deg,wsf2_m/s,wsf5_m/s
0,2016-01-01,5.7,0.0,0.0,0.0,5.0,6.1,1.1,270,280,11.6,15.7
1,2016-01-02,4.2,0.0,0.0,0.0,2.2,5.6,-1.1,260,260,9.4,11.2
2,2016-01-03,4.6,0.0,0.0,0.0,2.8,8.3,-2.2,270,250,10.7,13.4
3,2016-01-04,7.7,0.0,0.0,0.0,0.0,1.7,-10.0,330,330,11.6,14.8
4,2016-01-05,4.4,0.0,0.0,0.0,-7.2,-0.6,-12.2,360,350,11.2,13.9


## Sanity and physical check of the Data
***

Since there is no information on the level of checks of the data a quick sanity check is carried out to ensure all min and max values of each column in the dataset are within reasonable physical boundaries.

In [15]:
print(weather_df.describe())

         awnd_m/s     prcp_mm     snow_mm     snwd_mm   tavg_degc   tmax_degc  \
count  366.000000  366.000000  366.000000  366.000000  366.000000  366.000000   
mean     4.215574    2.669126    2.491530    8.703005   13.996721   18.885792   
std      1.675481    7.811253   32.423079   52.794662    9.703870   10.337672   
min      1.100000    0.000000    0.000000    0.000000  -13.300000   -7.800000   
25%      3.025000    0.000000    0.000000    0.000000    6.100000   10.725000   
50%      3.900000    0.000000    0.000000    0.000000   13.300000   18.900000   
75%      5.100000    0.800000    0.000000    0.000000   23.300000   28.300000   
max     10.200000   70.900000  609.600000  510.500000   31.700000   37.200000   

        tmin_degc    wdf2_deg   wdf5_deg    wsf2_m/s    wsf5_m/s  
count  366.000000  366.000000  366.00000  366.000000  366.000000  
mean     9.144262  217.841530  228.26776    9.159016   11.984153  
std      9.519682  102.548282   97.14852    3.063566    3.958846  
mi

# Exporting the Data Frame
***

In [16]:
weather_df.to_csv('C:/Users/Dell/weather.csv', index=False)

# Final Inspection
***

In [17]:
weather_df.head(5)

,date,awnd_m/s,prcp_mm,snow_mm,snwd_mm,tavg_degc,tmax_degc,tmin_degc,wdf2_deg,wdf5_deg,wsf2_m/s,wsf5_m/s
0,2016-01-01,5.7,0.0,0.0,0.0,5.0,6.1,1.1,270,280,11.6,15.7
1,2016-01-02,4.2,0.0,0.0,0.0,2.2,5.6,-1.1,260,260,9.4,11.2
2,2016-01-03,4.6,0.0,0.0,0.0,2.8,8.3,-2.2,270,250,10.7,13.4
3,2016-01-04,7.7,0.0,0.0,0.0,0.0,1.7,-10.0,330,330,11.6,14.8
4,2016-01-05,4.4,0.0,0.0,0.0,-7.2,-0.6,-12.2,360,350,11.2,13.9
